In [1]:
%cd /Users/natalipeeva/Documents/GitHub/Automatic-Answering-of-City-Council-Questions/

/Users/natalipeeva/Documents/GitHub/Automatic-Answering-of-City-Council-Questions


In [2]:
import sys
import os
src_dir = os.path.join(os.getcwd(), 'src')
sys.path.append(src_dir)

from read_data.read_data import read_urls_questions, get_questions, get_url_content_tuples, get_relevant_docs
from retrieval.sparse_retrieval.tfidf import perform_tfidf_search
from retrieval.evaluation.evaluate import calculate_recall_at_k, calculate_average_recall
import pandas as pd
from irmetrics.topk import recall, rr
import numpy as np

In [3]:
collected, questions =  read_urls_questions(os.path.join('data/reference_urls/reference_urls_collected.csv'),
                                            os.path.join('data/question_answer/questions_updated_urls.csv'), clean_url_nan=True) # read collected urls and questions + remove unsuccessful collection

amsterdam_only = questions[questions['Cleaned_URLs'].apply(lambda urls: any(url.startswith('https://www.amsterdam.nl') for url in urls))]
amsterdam_collected = pd.read_csv('data/collected/combined.csv')
amsterdam_references = collected[collected['URL'].str.startswith('https://www.amsterdam.nl')]
amsterdam_missing = pd.DataFrame()  # Empty DataFrame to store missing rows
missing_rows = []

for index, row in amsterdam_references.iterrows():
    if row['URL'] not in amsterdam_collected['URL'].tolist():
        missing_rows.append(row.to_dict())

amsterdam_missing = pd.DataFrame(missing_rows)
all_amsterdam = pd.concat([amsterdam_collected, amsterdam_missing])

In [4]:
question_list = get_questions(amsterdam_only)
document_list = get_url_content_tuples(all_amsterdam)

In [5]:
all_results = perform_tfidf_search(question_list, document_list, k=100) # took 5 minutes!!!!!!

In [12]:
question_list[0]

'\n7.\nKan het college de reeds bestaande zwemplekken in Amsterdam en de directe \nomgeving, die zich op fietsafstand bevinden, beter communiceren zodat mensen \nweten waar ze allemaal heen kunnen op de fiets om te zwemmen?\n'

In [17]:
all_results[list(all_results.keys())[0]][3][1]

'https://www.amsterdam.nl/veelgevraagd/?productid=%7BC9BE6316-1680-402C-A15E-91FA8D66BD82%7D'

In [8]:
relevant_docs = get_relevant_docs(questions)

predictions = []
for question in all_results.keys():
    urls = []
    for result in all_results[question]:
        urls.append(result[1])
    predictions.append(list(set(urls)))


true = []
for question in relevant_docs.keys():
    true.append(list(set(relevant_docs[question])))


# Calculate the Mean Reciprocal Rank for each question
mrr_values = []
for i in range(len(predictions)):
    true_values = true[i]
    mrr = rr(true_values, predictions[i], k=100)
    mrr_values.append(mrr)

# Calculate the average Mean Reciprocal Rank
average_mrr = np.mean(mrr_values)

print("Average Mean Reciprocal Rank:", average_mrr)


 # Calculate the Mean Reciprocal Rank for each question
mrr_values = []
for i in range(len(predictions)):
    true_values = true[i]
    mrr = recall(true_values, predictions[i], k=100)
    mrr_values.append(mrr)

# Calculate the average Mean Reciprocal Rank
average_recall = np.mean(mrr_values)

print("Average Recall:", average_recall)

Average Mean Reciprocal Rank: 0.00014269406392694063
Average Recall: 0.0136986301369863


### Passages

In [18]:
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk

# Download NLTK resources for Dutch tokenization
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/natalipeeva/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/natalipeeva/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /Users/natalipeeva/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [19]:
# Create an empty dataframe for the new data
new_df = pd.DataFrame(columns=['URL', 'Textual_Content'])

# Iterate over each row in the original dataframe
for index, row in all_amsterdam.iterrows():
    url = row['URL']
    content = str(row['Textual_Content'])
    
    # Tokenize the content into words
    words = word_tokenize(content, language='dutch')
    
    # Create passages of 100 words
    passages = [' '.join(words[i:i+100]) for i in range(0, len(words), 100)]
    
    # Create a new dataframe for the passages
    passage_df = pd.DataFrame({'URL': url, 'Textual_Content': passages})
    
    # Concatenate the new dataframe with the main dataframe
    new_df = pd.concat([new_df, passage_df], ignore_index=True)


In [20]:
question_list = get_questions(amsterdam_only)
document_list = get_url_content_tuples(new_df)
all_results = perform_tfidf_search(question_list, document_list, k=100) # took 5 minutes!!!!!!

In [21]:
relevant_docs = get_relevant_docs(questions)

predictions = []
for question in all_results.keys():
    urls = []
    for result in all_results[question]:
        urls.append(result[1])
    predictions.append(list(set(urls)))


true = []
for question in relevant_docs.keys():
    true.append(list(set(relevant_docs[question])))


# Calculate the Mean Reciprocal Rank for each question
mrr_values = []
for i in range(len(predictions)):
    true_values = true[i]
    mrr = rr(true_values, predictions[i], k=100)
    mrr_values.append(mrr)

# Calculate the average Mean Reciprocal Rank
average_mrr = np.mean(mrr_values)

print("Average Mean Reciprocal Rank:", average_mrr)


 # Calculate the Mean Reciprocal Rank for each question
mrr_values = []
for i in range(len(predictions)):
    true_values = true[i]
    mrr = recall(true_values, predictions[i], k=100)
    mrr_values.append(mrr)

# Calculate the average Mean Reciprocal Rank
average_recall = np.mean(mrr_values)

print("Average Recall:", average_recall)

Average Mean Reciprocal Rank: 0.0
Average Recall: 0.0


### Outcomes: 

Passages give 0 accuracy 
TF-IDF very bad